In [1859]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
import os
import pickle

from sklearn.ensemble import RandomForestClassifier

import pymysql
import json

config_fn = './config.json'


In [1860]:
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [1861]:
def connect(config):
    return pymysql.connect(
        host=config['ai_db_host'],  # Database host
        port=config['ai_db_port'],  # Database port
        user=config['ai_db_username'],  # Database user
        passwd=config['ai_db_password'],  # Database password
        db=config['ai_db_name'],  # Database name
        connect_timeout=5,
        cursorclass=pymysql.cursors.DictCursor
    )

def pull_data():
    with open(config_fn, "r") as f:
        config = json.loads(f.read())
    conn = connect(config)
    sql_1 = "SELECT rowId, question, category FROM cleanHotlineQuestionAnswer;"
    with conn.cursor() as cursor:
        cursor.execute(sql_1)
    result = cursor.fetchall()
    cursor.close()
    return result

In [1862]:
def cluster(df, df2, N, name, v=False):
    clusterer = KMeans(n_clusters=N)
    clusterer.fit(list(df.features))
    save_obj(clusterer, './models/clusterer_' + name )

    transform = clusterer.transform(list(df.features))

    d_center = []
    cluster = []
    for x in transform:
        d_center.append(min(x)**2)
        cluster.append(np.argmin(x))
    df['cluster'] = cluster
    df['d_from_center'] = d_center
    d_center = np.array(d_center)
    mean = np.mean(d_center)
    std = np.std(d_center)
    
    if v == True:
        print("Mean: {}".format(round(mean, 3)))
        print("STD: {}".format(round(std, 3)))
        print("")
         
        for cgroup in range(N):
            group = df.groupby('cluster').get_group(cgroup)
            print_clusters(group)

    return df

def print_clusters(group):
    std = np.std(list(group.d_from_center))
    mean = np.mean(list(group.d_from_center))
    
    center = group[group["d_from_center"] == min(group["d_from_center"])]

    center.drop_duplicates(subset=['question'], inplace=True)
    
    

    
    print("Found {} messages of the same form.  Mean: {} STD: {}".format(len(group), mean, std))
    print("*** {} ***".format(list(center.question)[0]))
    print("")
    for message in group.question.head(10):
        if group.question.count() > 1:
            print(message)
            print("")
    print("")

In [1863]:
def print_to_tsv(df, X, cat_name):
    vector_doc = './visualization_data/doc_vectors_' + cat_name + '.tsv'
    count = 0
    with open(vector_doc,'w') as w:
        for question in X:
            string = ""
            for v in question:
                string = string + str(v) + "\t"
            w.write(string + os.linesep)
            count += 1
    w.close
    print("Wrote file {} with {} entries".format(vector_doc, count))


    meta_doc = './visualization_data/doc_meta_' + cat_name + '.tsv'
    count = 0
    with open(meta_doc,'w') as w:
        w.write("cluster\tquestion\t" + os.linesep)
        for question, cluster in zip(list(df.question), list(df.cluster)):
            string = ""
            string = str(cluster) + "\t" + str(question) + "\t"
            w.write(string + os.linesep)  
            count += 1
    w.close
    print("Wrote file {} with {} entries".format(meta_doc, count))


In [1864]:
def train_model(df, N, name):
    print("Loaded {} Data Points".format(len(df)))
    vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.7 )
    X_vectoizer = vectorizer.fit_transform(list(df.question))
    save_obj(vectorizer, './models/vectorizer_' + name )
    print("Vectorization Complete")

    n_components = 60
    explained_variance = 0.0
    while explained_variance < .5 and n_components < 175:
        svd = TruncatedSVD(n_components=n_components)
        normalizer = Normalizer(copy=False)
        lsa = make_pipeline(svd, normalizer)
        X = lsa.fit_transform(X_vectoizer)
        
        save_obj(svd, './models/svd_' + name )
        save_obj(normalizer, './models/normalizer_' + name )
        df["features"] = list(X)
        
        explained_variance = svd.explained_variance_ratio_.sum()
        n_components += 5
    print("Explained variance of the SVD step: {}%     n_componets: {}".format(
        int(explained_variance * 100), n_components))
    df = cluster(df, X, N, name, v=True)
    print_to_tsv(df, X, name)

In [1865]:
def train_all():    
    df_master = pd.DataFrame(pull_data())
    cat_names = [ "Compliance", "Employee Benefits",
                "Leaves of Absence", "Recruiting and Hiring", "Terminations"]
    Ns = [ 15, 14, 9, 9, 7]

    for name, N in zip(cat_names, Ns):
        df = df_master[df_master["category"] == name].copy()
        train_model(df, N, name)
        df["prediction"] = predict(list(df.question), name)
        correct = len(df[df["cluster"] == df["prediction"]])
        total = float(len(df))
        print("The model {} consistantly classified {}% of the dataset".format(name, round(correct/total * 100, 1)))
        print("")






In [1866]:
def predict(messages, category):
    "This predicts the cluster of a message based on the kmeans model for the given category"
    "Inputs:"
    "        messages: a list of strings to be classified"
    "        category: a string containing the model to be used"
    "Output:"
    "        clusters: a list of intergers corresponding to the message classification cluster"
    
    vectorizer, svd, normalizer, clusterer = load_model(category)

    
    pipeline =  make_pipeline(vectorizer, svd, normalizer)
    messages = pipeline.transform(messages)
    
    clusters = clusterer.predict(messages)

    
    return clusters

def load_model(category):
    vectorizer = load_obj( './models/vectorizer_' + category )
    svd = load_obj( './models/svd_' + category )
    normalizer = load_obj( './models/normalizer_' + category )
    clusterer = load_obj("./models/clusterer_" + category)
    return vectorizer, svd, normalizer, clusterer

In [1867]:
train_all()



Loaded 27277 Data Points
Vectorization Complete
Explained variance of the SVD step: 50%     n_componets: 120
Mean: 0.687
STD: 0.15

Found 3690 messages of the same form.  Mean: 0.709560861424 STD: 0.101242823393
*** Thanks for the information. I have additional questions in regards to I-9. Could you please advise? * In our specific case, our headquarters (where all of the I9 Forms are stored) is located in NJ. However, we have stores in several different towns/cities throughout NY, NJ, PA, GA, and MA. How do we go about correcting Section 1 errors on the I9 form? * With the same case in mind, different managers filled out Section 2 of the I9. Are we (HR) allowed to fix the errors that they made or do they have to do it? If they have to do it, how do we go about it? What if they are currently inactive? * We have a lot that have errors so we made a spreadsheet of each active employee with I9 errors. We have also heard that it is advised against to ask employees to redo the I9 form (due t

C:\Users\kerri\Anaconda2\envs\thinkhr\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Found 3228 messages of the same form.  Mean: 0.674223515171 STD: 0.105559435401
*** WAND Corporation HR question - incentive attendance program with FMLA team member Company: WAND Corporation Think HR Hotline: Questions: 1) Can we exclude "John" from being eligible in the "SCURAP" program? 2) If not, can we ask him to remove himself from the program? 3) If not, how would you recommend we handle the below situation. Details: Our 24/7/365 call center was having issues with unplanned absences. To combat this, we rolled out an incentive program that employees are eligible for if they do not have a an unplanned absence for the period. We have one employee "John" who is on intermittent FMLA. He has several unplanned absences throughout the period which are identified as FMLA absences. For the first period we ran this program, FMLA unplanned absences did not exclude John from the incentive program. However, for this 2nd period (Q4-2017) of this program, he has had many unplanned FMLA absences

Hi Claudia, I should have clarified my inquiry it is in regards to the CA workers compensation requirement for first aid reporting. It appears there is a new amendment effective 1/1/2017 stating that first aid also needs to be reported for workers comp. https://www.wcirb.com/sites/default/files/bulletins/2016-25_reporting_small_medical_only_first_aid_claims.pdf However, on the ThinkHR website it says reporting is only for treatment beyond first aid

From : mary.pham@stp-sf.org Subject : Hello, I had a question about having a mother's room. Is a sink required?


Found 449 messages of the same form.  Mean: 0.59608345771 STD: 0.126084426535
*** If an employee has a temporary social security and then they bring in a permanent social security card, would the numbers be different? It is possible to get a new SSN? Is it possible to have an unrestricted SSN Card and a EAD card? We been doing E-Verify for two and she has been employed since 2013. Her EAD expires in November. ***

Thank you. If 

Found 1364 messages of the same form.  Mean: 0.713959482567 STD: 0.0994830151936
*** I am working on getting our personnel files compliant. We keep paper copies of files - nothing is filed electronically. In my research for best practices, I've found some concrete things (EEO-1, I-9, medical records, background/drug screens, etc. must be kept separate) but there is also a lot of grey area. Our corporate HR office has 3 people with access to personnel files and only 2 of those people handle record requests. Anyone requesting info is not handed the whole personnel file - instead, we screen and only provide pertinent information. Currently, our personnel file contains hiring/job docs, tax/payroll docs, and performance docs. We do keep those components separate by using tabbed dividers within the file. I worry that if we further break down the personnel file, we will run out of space. Because of the screening system we have in place, will it result in a compliance issue if we continue what

...

Found 443 messages of the same form.  Mean: 0.683032009152 STD: 0.104667410957
*** HI, we are implementing background checks at our organization and need guidance on order of process. Do we have to offer the job and receive acceptance before we run the background check? or do can we order the background check with the proper authorization and offer the job after the background has been ran? ***

We currently do not do background checks, but we are considering implementing them. Can we set a hard and fast rule that candidates can have no felonies whatsoever to pass the background process?

What forms are we required to give a job candidate that is rejected due to their background check? We have locations in CA, NY, OR, MA, PA, VA, FL, CT, NH

When we issue a candidate a preliminary adverse action letter based upon background check results, how long do we have to give him to respond before taking final action?

Hello! Should I be stating the fact that we do background checks and pre empl

Found 821 messages of the same form.  Mean: 0.694350591754 STD: 0.0972266536123
*** References Hi, We had a potential employer for a terminated employee call one of our supervisors on the company number to ask them for "Professional References". Normally, I take all Employment verification calls but this really wasn't that type of call. How would we handle this call? From my experience we aren't supposed to give opinions on job performance, work ethic or any of those things. Can you please advise on what we should do in this case? Should they have called this person on their personal phone to ask these questions???? Have a fantastic day!!! [Carleton Gotlin] DAWN J DEPRIEST OFFICE MANAGER Carleton Gotlin Law PC 1580 Lincoln St, Suite 1200 Denver, CO 80203 T: 303.825.1125 F: 303.302.3088 www.carletongotlinlaw.com NOTICE: The information set forth in this electronic mail transmission, and any attachments, are intended solely for the person to whom it is addressed, and may contain confiden

The model Recruiting and Hiring consistantly classified 100.0% of the dataset

Loaded 6830 Data Points
Vectorization Complete
Explained variance of the SVD step: 50%     n_componets: 110
Mean: 0.699
STD: 0.149

Found 833 messages of the same form.  Mean: 0.834745727724 STD: 0.104611686039
*** Confidential. ***

Do you have access to or have any benchmark data for executive employment agreements as it relates to severance payments. Specifically for a company with around 175 employees in Tennessee. I found a Lee Hecht Harrison and a world at work study. It's difficult to compare to those since company size is important.

Are there any recent (last 24 months) severance agreement cases that provided favorable outcomes to employees with under 25 employees? Are severance agreements advised for groups that size if they have a clear employee handbook signed at hire? What states require severance?

Do you have a proprietary data form for exiting employees to sign? Currently have a non-disclosur

Found 1275 messages of the same form.  Mean: 0.563866922643 STD: 0.134597632781
*** Hello, We have a situation that Id like some guidance on: Our receptionist has been feeling stressed about work and life issues. In high emotion on 1/14/17 she said she was packing her desk and quitting. Our General Manager encouraged her to take a week of personal time to decide if she wanted to stay and work things out, or if she really wanted to quit. She did take her personal items with her that day. While she was gone, the Club functioned better than usual, without the emotion and stress that her presence created. We hoped she would decide to quit, but planned to terminate her otherwise. We called her today before she reported for her return shift and she informed us that she did indeed decide to retire. She intended to come in to work out her notice, but we told her that we think it is best to end things immediately and we would pay her for the two weeks in lieu of her coming in. We have not done 

Wrote file ./visualization_data/doc_vectors_Terminations.tsv with 6830 entries
Wrote file ./visualization_data/doc_meta_Terminations.tsv with 6830 entries
The model Terminations consistantly classified 100.0% of the dataset

